### Imports

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00


In [2]:
from typing import List, Tuple

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import spacy
from datasets import load_dataset

### Load Data & Normalize text

In [4]:
# Function for load dataset Davlan/sib200 (rus_Cyrl)

def load_sib200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    # Load dataset
    dataset = load_dataset("Davlan/sib200", "rus_Cyrl")

    print(dataset)

    # Need to encode categories
    train = dataset['train'].class_encode_column('category')
    valid = dataset['validation'].class_encode_column('category')
    test = dataset['test'].class_encode_column('category')

    # Check categories
    categories = train.features['category'].names
    if  categories != valid.features['category'].names or \
        categories != test.features['category'].names:
        raise RuntimeError("Categories don't match!")

    X_train, y_train = train['text'], train['category']
    X_val, y_val = valid['text'], valid['category']
    X_test, y_test = test['text'], test['category']

    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

In [5]:
# Normalize text
def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    # String to lower
    s = s.lower()

    # Processing text with the NLP Pipeline
    doc = nlp_pipeline(s)

    # Lemmatization and replacement of numbers by <NUM>
    lemmas = []
    for token in doc:
        if token.is_punct:
            continue                        # Skip punctuation
        if token.like_num:
            lemmas.append('<NUM>')          # Replace numeric tokens with <NUM>
        else:
            lemmas.append(token.lemma_)     # Adding a token lemma

    return ' '.join(lemmas)

In [6]:
# Function for unpacking Tuple for normalize_text
def process_tuple(data: Tuple[List[str], List[int]], nlp_pipeline: spacy.Language) -> Tuple[List[str], List[int]]:
    texts, categories = data
    normalized_texts = [normalize_text(text, nlp_pipeline) for text in texts]
    return (normalized_texts, categories)

In [7]:
train_data, val_data, test_data, classes_list = load_sib200_ru()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

data/rus_Cyrl/train.tsv:   0%|          | 0.00/195k [00:00<?, ?B/s]

data/rus_Cyrl/dev.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

data/rus_Cyrl/test.tsv:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

...

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 701
    })
    validation: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 99
    })
    test: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 204
    })
})


Casting to class labels:   0%|          | 0/701 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/99 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/204 [00:00<?, ? examples/s]

In [8]:
# Print category list
print(classes_list)

['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [9]:
# Check the dimensionality of all obtained samples
print('Train shape: {}'.format(len(train_data[0])))
print('Validation shape: {}'.format(len(val_data[0])))
print('Test shape: {}'.format(len(test_data[1])))

Train shape: 701
Validation shape: 99
Test shape: 204


In [10]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
# Load pipeline ru_core_news_sm
nlp = spacy.load('ru_core_news_sm')

In [12]:
# Apply normalize_text and nlp to all samples
train_data = process_tuple(train_data, nlp)
val_data = process_tuple(val_data, nlp)
test_data = process_tuple(test_data, nlp)

### Init & Fit model (LogReg)

In [13]:
# Create Pipeline for text vectorization and logistic regression
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('estimator', LogisticRegression())
])

In [14]:
# Parameterize using GridSearchCV
param_grid = {"estimator__C": np.linspace(0.1, 1000, 5),
              "estimator__tol": [1e-1, 1e-2, 1e-4, 1e-6],
              "estimator__solver": ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']}

LR_search = GridSearchCV( estimator=pipeline,
                          param_grid=param_grid,
                          scoring='f1_macro',
                          n_jobs = -1 )

LR_search.fit(train_data[0], train_data[1])

GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('estimator', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'estimator__C': array([1.00000e-01, 2.50075e+02, 5.00050e+02, 7.50025e+02, 1.00000e+03]),
                         'estimator__solver': ['lbfgs', 'liblinear',
                                               'newton-cg', 'newton-cholesky',
                                               'sag', 'saga'],
                         'estimator__tol': [0.1, 0.01, 0.0001, 1e-06]},
             scoring='f1_macro')

In [15]:
# Print the best parameters
LR_search.best_params_

{'estimator__C': 500.05,
 'estimator__solver': 'newton-cholesky',
 'estimator__tol': 0.01}

### Score & Results

In [16]:
# Print the best F1 score
LR_search.best_score_

0.6539082145218936

In [17]:
# Print the size of vectorized text dictionary
LR_search.best_estimator_.named_steps['tfidf'].vocabulary_.__len__()

4330

In [18]:
# Get predictions on the validation sample and output classification_report
val_pred = LR_search.predict(val_data[0])
print(classification_report(val_data[1], val_pred))

              precision    recall  f1-score   support

           0       0.83      0.56      0.67         9
           1       0.67      0.75      0.71         8
           2       1.00      0.55      0.71        11
           3       0.90      0.64      0.75        14
           4       0.62      0.80      0.70        25
           5       0.77      0.83      0.80        12
           6       0.57      0.65      0.60        20

    accuracy                           0.70        99
   macro avg       0.77      0.68      0.70        99
weighted avg       0.73      0.70      0.70        99



In [19]:
# Get predictions on the test sample and output classification_report
test_pred = LR_search.predict(test_data[0])
print(classification_report(test_data[1], test_pred))

              precision    recall  f1-score   support

           0       0.78      0.37      0.50        19
           1       0.67      0.59      0.62        17
           2       0.48      0.50      0.49        22
           3       0.78      0.83      0.81        30
           4       0.66      0.80      0.73        51
           5       0.90      0.76      0.83        25
           6       0.60      0.62      0.61        40

    accuracy                           0.68       204
   macro avg       0.70      0.64      0.65       204
weighted avg       0.69      0.68      0.67       204

